In [1]:
spark.sql("show schemas").show()

In [2]:
dfStatic = spark.read\
.format("csv")\
.option("header","true")\
.option("inferSchema","true")\
.load("/FileStore/tables/retailDate.csv")

staticSchema = dfStatic.schema
staticSchema

In [3]:
#1. create stream DF
dfStream = spark.readStream\
.option("masFilesPerTrigger", 1)\
.format("csv")\
.option("header","true")\
.schema(staticSchema)\
.load("/FileStore/tables/*.csv")\

In [4]:
dfStream.isStreaming

In [5]:
#2. some code to process straming data
from pyspark.sql.functions import col, window
dfNew = dfStream.selectExpr("CustomerID"
                           , "(UnitPrice * Quantity) as total_cost"
                           ,"InvoiceDate")\
                           .groupBy( col("CustomerID"), window(col("InvoiceDate"),"1 day"))\
                            .sum("total_cost")
                           

In [6]:
#3. start writing stream to memory 
dfNew.writeStream\
.format("memory")\
.queryName("test")\
.outputMode("complete")\
.start()

In [7]:
#4. read from memory
spark.sql("select * from test").show()

In [8]:
dfNew.writeStream\
.format("console")\
.queryName("test_2")\
.outputMode("complete")\
.start()


#Neither of these streaming methods should be used in production 
